## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [139]:
%%sql
postgresql:///animal_shelters
    
    WITH PREP1 AS (
        
    SELECT
        *
        , d.costs as size_cost
        , d.animaltype as animaltype_d
        , a.animaltype as animaltype_a
        , d.size as size_d
        , a.size as size_a
    FROM
    (
    SELECT 
        *
        , animalid
        
        -- flag animal sizy by type and weight
        , CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'small'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'small'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'small' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'medium'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'medium'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'medium' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'large'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'large'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'large' 
          END AS size
        , RIGHT(birthdate, 4) AS BIRTH_YEAR
        , CAST(cast(DATE_PART('year',AGE('2021-12-31',cast(birthdate as date))) as int) AS char)  AS AGE_LEFT
        , b.costs as AGE_COST
        , c.costs AS LOCATION_COST
        , (UPPER(LEFT(animaltype,1)) || (CASE
                                                    -- small animal
                                                    WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                                                        THEN 'S'
                                                    WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                                                        THEN 'S'  
                                                    WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                                                        THEN 'S' 
                
                                                    -- medium animal
                                                        WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                                                        THEN 'M'
                                                        WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                                                        THEN 'M'  
                                                    WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                                                        THEN 'M' 
                
                                                    -- large animal
                                                    WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                                                        THEN 'L'
                                                    WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                                                        THEN 'L'  
                                                    WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                                                        THEN 'L' 
                                                 END)
          ) AS sizeid
        , A

    FROM 
        animals as a
    LEFT JOIN 
        age_costs as b
    ON 
        CAST(cast(DATE_PART('year',AGE('2021-12-31',cast(birthdate as date))) as int) AS char) = b.age
    LEFT JOIN
        location_costs as c
    ON 
        a.location = c.location
    WHERE 
        animalid NOT IN (SELECT DISTINCT
                             sponsorid
                         FROM 
                             sponsored_animals)
    ) AS a
       LEFT JOIN
       size_costs AS d
    ON
      a.sizeid = d.sizeid
    ) 
    
    
    
    
    SELECT 
--        lower(animaltype_a) AS animaltype
--        , size_a as size
--        , count(*) as count
         SUM(age_cost) AS AGE_COST
        , SUM(location_cost) AS LOCATION_COST
        , SUM(size_cost) AS SIZE_COST
        ,  SUM(age_cost + location_cost + size_cost) AS TOTAL
        , (SELECT SUM(age_cost) + SUM(location_cost) +  SUM(size_cost) FROM PREP1) gran_total
        , ROUND((SUM(age_cost) + SUM(location_cost) +  SUM(size_cost)) / ((SELECT SUM(age_cost) + SUM(location_cost) +  SUM(size_cost) FROM PREP1) * 0.01), 2) AS PERCENTAGE
    FROM 
        PREP1
--    GROUP BY
--        animaltype_a
--        ,size_a      
--    ORDER BY
--        animaltype_a ASC
--        , size_a desc

1 rows affected.


age_cost,location_cost,size_cost,total,gran_total,percentage
585150,1216325,1375260,3176735,3176735,100.00


In [140]:
%%sql
postgresql:///animal_shelters
with a as (    
    select 
        *
        , CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'small'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'small'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'small' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'medium'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'medium'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'medium' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'large'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'large'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'large' 
          END AS size
        , left(UPPER(animaltype), 1) ||  
            CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'S'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'S'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'S' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'M'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'M'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'M' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'L'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'L'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'L' 
          END as sizeid
        , cast(DATE_PART('year',AGE('2021-12-31',cast(birthdate as date))) as int) AS AGE
        , DATE_PART('year', '2021-12-31'::date) - DATE_PART('year', birthdate::date) as datediff
    from
        animals
    where
        animalid not in (select distinct sponsorid from sponsored_animals)
    )


    select sizeid, SUM(AGE), sum(datediff) as age from a group by sizeid order by age asc

9 rows affected.


sizeid,sum,age
BS,27,27.0
BM,54,54.0
BL,141,141.0
CM,4448,4448.0
DS,6706,6706.0
CL,7262,7262.0
CS,9814,9814.0
DL,17277,17277.0
DM,17889,17889.0


In [141]:
%%sql
postgresql:///animal_shelters
    with s as (
    select distinct sponsorid from sponsored_animals
    )
    select * from s

2142 rows affected.


sponsorid
e7eae8cc0dbde1
e1259b946c13b4
814d404bef56de
82f6e848080cf4
cb763922b3f64e
b9b57fbcb2170b
e5ea3e7de3ede7
f9985aeb87885a
5b0cee68f6a848
0a404486cecf34


In [142]:
%%sql
postgresql:///animal_shelters
    with l as (
    select location, costs as location_cost from location_costs
    )
    select * from l

5 rows affected.


location,location_cost
San Antonio,120
Dallas,110
Houston,140
Austin,135
Fort Worth,100


In [143]:
%%sql
postgresql:///animal_shelters
with a as (    
    select 
        *
        , CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'small'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'small'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'small' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'medium'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'medium'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'medium' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'large'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'large'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'large' 
          END AS size
        , left(UPPER(animaltype), 1) ||  
            CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'S'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'S'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'S' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'M'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'M'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'M' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'L'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'L'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'L' 
          END as sizeid
        , cast(DATE_PART('year',AGE('2021-12-31',cast(birthdate as date))) as int) AS AGE
        , DATE_PART('year', '2021-12-31'::date) - DATE_PART('year', birthdate::date) as datediff
    from
        animals a
    left join
        (select location, costs as location_cost from location_costs) as l
    on
        a.location = l.location
    where
        animalid not in (select distinct sponsorid from sponsored_animals)
    )


    select 
        sum(location_cost) as location_cost 
    from a

1 rows affected.


location_cost
1216325


In [144]:
%%sql
postgresql:///animal_shelters
    with l as (
    select sizeid, costs as size_cost from size_costs
    )
    select * from l

9 rows affected.


sizeid,size_cost
DL,175
DM,150
DS,125
CL,140
CM,120
CS,100
BL,110
BM,90
BS,70


In [145]:
%%sql
postgresql:///animal_shelters
with a as (    
    select 
        *
        , CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'small'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'small'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'small' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'medium'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'medium'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'medium' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'large'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'large'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'large' 
          END AS size
        , left(UPPER(animaltype), 1) ||  
            CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'S'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'S'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'S' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'M'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'M'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'M' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'L'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'L'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'L' 
          END as sizeid
        , cast(DATE_PART('year',AGE('2021-12-31',cast(birthdate as date))) as int) AS AGE
        , DATE_PART('year', '2021-12-31'::date) - DATE_PART('year', birthdate::date) as datediff
    from
        animals a
    left join
        (select location, costs as location_cost from location_costs) as l
    on
        a.location = l.location
    left join
        (select sizeid, costs as size_cost from size_costs) as s
    on
        left(UPPER(animaltype), 1) ||  
            CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'S'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'S'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'S' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'M'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'M'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'M' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'L'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'L'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'L' 
          END = s.sizeid
    where
        animalid not in (select distinct sponsorid from sponsored_animals)
    )


    select 
        sum(location_cost) as location_cost 
        ,sum(size_cost) as size_cost 
    from a

1 rows affected.


location_cost,size_cost
1216325,1375260


In [146]:
%%sql
postgresql:///animal_shelters

    select age as age_ac, costs as age_cost from age_costs

51 rows affected.


age_ac,age_cost
0,0
1,0
2,0
3,0
4,0
5,100
6,100
7,100
8,150
9,150


In [147]:
%%sql
postgresql:///animal_shelters
with a as (    
    select 
        *
        , CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'small'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'small'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'small' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'medium'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'medium'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'medium' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'large'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'large'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'large' 
          END AS size
        , left(UPPER(animaltype), 1) ||  
            CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'S'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'S'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'S' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'M'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'M'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'M' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'L'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'L'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'L' 
          END as sizeid
        , cast(DATE_PART('year', TO_DATE('12/31/2021', 'MM/DD/YYYY')) - DATE_PART('year', TO_DATE(birthdate, 'MM/DD/YYYY')) as int) as age     

    from
        animals a
    left join
        (select location, costs as location_cost from location_costs) as l
    on
        a.location = l.location
    left join
        (select sizeid, costs as size_cost from size_costs) as s
    on
        left(UPPER(animaltype), 1) ||  
            CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'S'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'S'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'S' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'M'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'M'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'M' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'L'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'L'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'L' 
          END = s.sizeid
    left join
        (select age as age_ac, costs as age_cost from age_costs) as ac
    on
        cast(DATE_PART('year', TO_DATE('12/31/2021', 'MM/DD/YYYY')) - DATE_PART('year', TO_DATE(birthdate, 'MM/DD/YYYY')) as int) = cast(ac.age_ac as int)
    where
        animalid not in (select distinct sponsorid from sponsored_animals)
    )

    select 
--        age
        sum(location_cost) as location_cost 
        ,sum(size_cost) as size_cost 
        ,sum(age_cost) as age_cost
    from a
--    group by age
--    order by age

1 rows affected.


location_cost,size_cost,age_cost
1216325,1375260,885450


In [148]:
%%sql
postgresql:///animal_shelters
    SELECT SUM(AGE) AS AGE , SUM(CAST(AGE_AC AS INT)) AS AGEAC
    FROM
    (
    select distinct 
     TO_DATE('31/12/2021', 'DD/MM/YYYY') as today,
     TO_DATE(birthdate, 'MM/DD/YYYY') as birthdate,
    birthdate as birthdate_original,
     cast(DATE_PART('year', TO_DATE('12/31/2021', 'MM/DD/YYYY')) - DATE_PART('year', TO_DATE(birthdate, 'MM/DD/YYYY')) as int) as age     
    ,b.age as age_ac
    from animals a
    left join age_costs b
    on  cast(DATE_PART('year', TO_DATE('12/31/2021', 'MM/DD/YYYY')) - DATE_PART('year', TO_DATE(birthdate, 'MM/DD/YYYY')) as int) = cast(b.age as int)
   -- where cast(DATE_PART('year', TO_DATE('12/31/2021', 'MM/DD/YYYY')) - DATE_PART('year', TO_DATE(birthdate, 'MM/DD/YYYY')) as int) != cast(b.age as int)
    order by age
    
    ) A
    

1 rows affected.


age,ageac
33316,33316


In [149]:
%%sql
postgresql:///animal_shelters
with a as (    
    select 
        *
        , CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'small'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'small'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'small' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'medium'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'medium'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'medium' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'large'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'large'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'large' 
          END AS size
        , left(UPPER(animaltype), 1) ||  
            CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'S'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'S'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'S' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'M'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'M'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'M' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'L'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'L'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'L' 
          END as sizeid
        , cast(DATE_PART('year', TO_DATE('12/31/2021', 'MM/DD/YYYY')) - DATE_PART('year', TO_DATE(birthdate, 'MM/DD/YYYY')) as int) as age     

    from
        animals a
    left join
        (select location, costs as location_cost from location_costs) as l
    on
        a.location = l.location
    left join
        (select sizeid, costs as size_cost from size_costs) as s
    on
        left(UPPER(animaltype), 1) ||  
            CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'S'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'S'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'S' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'M'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'M'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'M' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'L'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'L'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'L' 
          END = s.sizeid
    left join
        (select age as age_ac, costs as age_cost from age_costs) as ac
    on
        cast(DATE_PART('year', TO_DATE('12/31/2021', 'MM/DD/YYYY')) - DATE_PART('year', TO_DATE(birthdate, 'MM/DD/YYYY')) as int) = cast(ac.age_ac as int)
    where
        animalid not in (select distinct sponsorid from sponsored_animals)
    )

    select 
--        age
        sum(location_cost) as location_cost 
        ,sum(size_cost) as size_cost 
        ,sum(age_cost) as age_cost
    from a
--    group by age
--    order by age

1 rows affected.


location_cost,size_cost,age_cost
1216325,1375260,885450


In [150]:
%%sql
postgresql:///animal_shelters
with a as (    
    select 
        *
        , CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'small'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'small'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'small' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'medium'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'medium'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'medium' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'large'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'large'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'large' 
          END AS size
        , left(UPPER(animaltype), 1) ||  
            CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'S'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'S'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'S' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'M'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'M'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'M' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'L'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'L'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'L' 
          END as sizeid
        , cast(DATE_PART('year', TO_DATE('12/31/2021', 'MM/DD/YYYY')) - DATE_PART('year', TO_DATE(birthdate, 'MM/DD/YYYY')) as int) as age     

    from
        animals a
    left join
        (select location, costs as location_cost from location_costs) as l
    on
        a.location = l.location
    left join
        (select sizeid, costs as size_cost from size_costs) as s
    on
        left(UPPER(animaltype), 1) ||  
            CASE
            -- small animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 10)
                THEN 'S'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 5)
                THEN 'S'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 0.7)
                THEN 'S' 
                
            -- medium animal
            WHEN (UPPER(animaltype) = 'DOG') AND (weight <= 30)
                THEN 'M'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight <= 7)
                THEN 'M'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight <= 1.1)
                THEN 'M' 
                
            -- large animal
                        WHEN (UPPER(animaltype) = 'DOG') AND (weight > 30)
                THEN 'L'
            WHEN (UPPER(animaltype) = 'CAT') AND (weight > 7)
                THEN 'L'  
            WHEN (UPPER(animaltype) = 'BIRD') AND (weight > 1.1)
                THEN 'L' 
          END = s.sizeid
    left join
        (select age as age_ac, costs as age_cost from age_costs) as ac
    on
        cast(DATE_PART('year', TO_DATE('12/31/2021', 'MM/DD/YYYY')) - DATE_PART('year', TO_DATE(birthdate, 'MM/DD/YYYY')) as int) = cast(ac.age_ac as int)
    where
        animalid not in (select distinct sponsorid from sponsored_animals)
    )



    SELECT 
        animaltype
        , size
        , SUM(age_cost + location_cost + size_cost) AS TOTAL
--        , (SELECT SUM(age_cost) + SUM(location_cost) +  SUM(size_cost) FROM a) gran_total
        , ROUND((SUM(age_cost) + SUM(location_cost) +  SUM(size_cost)) / ((SELECT SUM(age_cost) + SUM(location_cost) +  SUM(size_cost) FROM a) * 0.01), 2) AS PERCENTAGE
    FROM 
        a
    group by
        animaltype
        , size
    order by
        animaltype
        , size desc
    

9 rows affected.


animaltype,size,total,percentage
Bird,small,1615,0.05
Bird,medium,3130,0.09
Bird,large,8120,0.23
Cat,small,518015,14.90
Cat,medium,250575,7.21
Cat,large,439490,12.64
Dog,small,336530,9.68
Dog,medium,941895,27.09
Dog,large,977665,28.12
